In [16]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

In [14]:
#!pip install implicit

     -------------------------------------- 633.2/633.2 kB 2.1 MB/s eta 0:00:00


In [25]:
#!pip install h5py

  Using cached h5py-3.7.0-cp310-cp310-win_amd64.whl (2.6 MB)


In [17]:
#считывание данных
users = pd.read_csv('train/users.csv', sep=';', index_col=None, dtype={'age': str, 'chb': str, 'chit_type': str, 'gender': str})
items = pd.read_csv('train/items.csv', sep=';', index_col=None, dtype={'author': str, 'bbk': str, 'izd': str, 'sys_numb': str, 'title': str, 'year_izd': str})
train_transactions = pd.read_csv('train/train_transactions_extended.csv', sep=';', index_col=None, dtype={'chb': str, 'date_1': str, 'is_printed': str, 'is_real': str, 'source': str, 'sys_numb': str, 'type': str})

In [18]:
print(f"Кол-во пользователей: {len(train_transactions['chb'].unique())}")
print(f"Кол-во документов в истории пользователей: {len(train_transactions['sys_numb'].unique())}")
print(f"Общее кол-во документов: {len(items['sys_numb'].unique())}")

Кол-во пользователей: 16753
Кол-во документов в истории пользователей: 194666
Общее кол-во документов: 354355


In [5]:
#строго фиксируем кол-во пользователей и уникальных документов
n_users = len(train_transactions['chb'].unique())
n_items = len(items['sys_numb'].unique())

In [6]:
#т.к далее придётся работать с матрицами создадим словари, которые точно отображают индексы в chb/sys_numb и обратно
mapping_chb_index = {chb_number: index for index, chb_number in enumerate(train_transactions['chb'].unique())}
mapping_sys_numb_index = {sys_number: index for index, sys_number in enumerate(items['sys_numb'].unique())}

mapping_index_chb = {index: chb_number for index, chb_number in enumerate(train_transactions['chb'].unique())}
mapping_index_sys_numb = {index: sys_number for index, sys_number in enumerate(items['sys_numb'].unique())}

In [7]:
#в базовом решении будем не будем использовать дополнительные данные о взаимодействиях
train_transactions = train_transactions[['chb', 'sys_numb']]

In [8]:
from sklearn.model_selection import train_test_split

# делим данные на тренировочный и тестовый наборы
train_data, test_data = train_test_split(train_transactions, test_size=0.20)

In [24]:
# Не очень удачное разбиение на train и test, поскольку в выборке для тестирования присутствуют не все пользователи. 
# Это означает, что понять, насколько рекомендательна система качественно работает для данного сегмента не получится.

print(f"Кол-во уникальных пользователей: {len(train_transactions['chb'].unique())}")
print(f"Кол-во уникальных пользователей в выборке для обучения: {len(train_data['chb'].unique())}")
print(f"Кол-во уникальных пользователей в выборке для тестирования: {len(test_data['chb'].unique())}")

Кол-во уникальных пользователей: 16753
Кол-во уникальных пользователей в выборке для обучения: 16675
Кол-во уникальных пользователей в выборке для тестирования: 12379


In [11]:
# Имеем дело с разряженными матрицами с ними лучше работать в sparse формате

def df_to_sparse(df):
    row = []
    col = []
    data = []

    for line in df.itertuples():
        row.append(mapping_chb_index[line.chb])
        col.append(mapping_sys_numb_index[line.sys_numb])
        data.append(1)

    return csr_matrix((data, (row, col)))

In [ ]:
# Имеем дело с разряженными матрицами с ними лучше работать в sparse формате

def df_to_sparse0(df):
    row = []
    col = []
    data = []

    for line in df.itertuples():
        row.append(mapping_chb_index[line.chb])
        cur_sys_numb = line.sys_numb
        col.append(mapping_sys_numb_index[line.sys_numb])
        data.append(1)

    return csr_matrix((data, (row, col)))

In [12]:
# получение sparse матрицы user-item для train/test
train_data_sparse = df_to_sparse(train_data)
test_data_sparse = df_to_sparse(test_data)

In [48]:
from implicit.als import AlternatingLeastSquares

#В качестве базового решения попробуем алгоритм ALS. Подробнее можно ознакомиться с ним здесь - https://github.com/benfred/implicit
model = AlternatingLeastSquares()
model.fit(train_data_sparse)

  0%|          | 0/15 [00:00<?, ?it/s]

In [26]:
# Получим рекомендации для конкретного пользователя 
userid = 2233
ids, scores = model.recommend(userid, train_data_sparse[userid], N=20, filter_already_liked_items=True)

In [27]:
ids, scores

(array([201127, 164122,  87192, 223024, 108298,  85490, 262997, 272421,
        253593, 259039, 276186, 214956, 247717, 189785, 155667,  20479,
        251169, 206303, 126120, 336502]),
 array([3.5023299e-05, 3.4865916e-05, 3.4673121e-05, 3.4142184e-05,
        3.3430835e-05, 3.3094668e-05, 3.2136042e-05, 3.1513784e-05,
        3.1461524e-05, 3.1351825e-05, 3.1100913e-05, 3.1026342e-05,
        3.1010091e-05, 3.0926440e-05, 3.0921947e-05, 3.0899020e-05,
        3.0893709e-05, 3.0893709e-05, 3.0889249e-05, 3.0878971e-05],
       dtype=float32))

In [36]:
tsys_numb = []
for id in ids:
    tsys_numb.append(mapping_index_sys_numb[id])
    print(mapping_index_sys_numb[id])

RSL01009646136
RSL01004125780
RSL01002679335
RSL01007046521
RSL01007830458
RSL01002463969
RSL01010448074
RSL01002835455
RSL01002835160
RSL01010677200
RSL01008959412
RSL01010323553
RSL01010544270
RSL01010313840
RSL01002792974
RSL01002993589
RSL01010528416
RSL01010185166
RSL01005462641
RSL01002958891


In [30]:
tuserid = mapping_index_chb[userid]

In [33]:
tids = train_transactions[train_transactions['chb']==tuserid]

In [49]:
# Отобразим рекомендации в DataFrame
top20recom_df = pd.DataFrame({"sys_numb": [mapping_index_sys_numb[id] for id in ids], "score": scores, "already_liked": np.in1d(ids, train_data_sparse[userid].indices)})

In [50]:
def get_recom(userid):
    ids, scores = model.recommend(userid, train_data_sparse[userid], N=20, filter_already_liked_items=True)
    top20recom_df = pd.DataFrame({"sys_numb": [mapping_index_sys_numb[id] for id in ids], "score": scores, "already_liked": np.in1d(ids, train_data_sparse[userid].indices)})
    return top20recom_df['sys_numb'].values

In [51]:
#подбор рекомендаций для всех пользователей из train

all_rec = []

for userid in tqdm(range(train_data_sparse.shape[0])):
    user_chb = mapping_index_chb[userid]
    user_rec = get_recom(userid)
    for rec in user_rec:
        all_rec.append([user_chb, rec])

  0%|          | 0/16753 [00:00<?, ?it/s]

In [52]:
# DataFrame для отправки решения с рекомендациями
solution = pd.DataFrame(all_rec, columns=["chb", "sys_numb"])

In [53]:
# Формирование csv файла для отправки на платформу
solution.to_csv("solution.csv", index=False, sep=';')

## Метрика

In [ ]:
df_solution = solution
df_grd = test_data

In [ ]:
#считаем recall, precision, f1_score

def metric(df_solution, df_grd):
    pred = set(df_solution['chb'] + '_' + df_solution['sys_numb'].values)
    true = set(df_grd['chb'] + '_' + df_grd['sys_numb'].values)
    recall = len(pred.intersection(true)) / len(true)
    precision = len(pred.intersection(true)) / (20 * len(df_grd['chb'].unique()))
    f1_score = 2 * (precision * recall) / (precision + recall)
    print(f"Recall: {round(recall, 5)}")
    print(f"Precision: {round(precision, 5)}")
    print(f"F1-score: {round(f1_score, 5)}")

In [ ]:
# оценка базового решения

metric(df_solution, df_grd)

Recall: 0.01527
Precision: 0.00309
F1-score: 0.00514


In [ ]:
# подадим ответы в качестве рекомендаций
metric(df_grd, df_grd)

Recall: 1.0
Precision: 0.20222
F1-score: 0.33641


In [ ]:
# создадим случайные рекомендации 
random_solution = []
for chb in tqdm(set(df_grd['chb'].values)):
    for sys in items['sys_numb'].sample(20).values:
        random_solution.append([chb, sys])

  0%|          | 0/12406 [00:00<?, ?it/s]

In [ ]:
# оценка подхода со случайными рекомендациями
metric(pd.DataFrame(random_solution, columns=['chb', 'sys_numb']), df_grd)

Recall: 4e-05
Precision: 1e-05
F1-score: 1e-05


### Занимаемся BM25

In [1]:
!pip install rank_bm25

In [13]:
from rank_bm25 import BM25Okapi

corpus = [
    "Привет моя дорогая"
    "Hello there good man!",
    "It is quite windy in London",
    "How is the weather today?"
]

tokenized_corpus = [doc.split(" ") for doc in corpus]

bm25 = BM25Okapi(tokenized_corpus)

In [14]:
query = "Привет"
tokenized_query = query.split(" ")

doc_scores = bm25.get_scores(tokenized_query)

In [15]:
doc_scores

array([0.49765247, 0.        , 0.        ])

In [24]:
items['title']

0         Судебное следствие в уголовном процессе России...
1         Уральское казачество и его роль в системе росс...
2                                               отсутствует
3                                               отсутствует
4         "Я пишу как эхо другого.." : Очерки интеллекту...
                                ...                        
354350                 Канада. География, экономика, право 
354351    Изменения географических названий стран СНГ : ...
354352    К информационному обществу: использование инфо...
354353    Один раз на всю жизнь : возможен ли брак по лю...
354354    Works / Min. of culture a. monuments protectio...
Name: title, Length: 354355, dtype: object

In [82]:
corpus = items['title'].tolist()

In [83]:
for i in range(len(corpus)):
    corpus[i] = corpus[i].strip()
    corpus[i] = corpus[i].lower()
    corpus[i] = corpus[i].replace(':','')
    corpus[i] = corpus[i].replace('/','')
    
tokenized_corpus = [doc.split() for doc in corpus]

In [84]:
tokenized_corpus

[['судебное',
  'следствие',
  'в',
  'уголовном',
  'процессе',
  'россии',
  'монография'],
 ['уральское',
  'казачество',
  'и',
  'его',
  'роль',
  'в',
  'системе',
  'российской',
  'государственности',
  '(середина',
  'xvii',
  '-',
  'xix',
  'вв.)',
  'диссертация',
  '..',
  'доктора',
  'исторических',
  'наук',
  '07.00.02'],
 ['отсутствует'],
 ['отсутствует'],
 ['"я',
  'пишу',
  'как',
  'эхо',
  'другого.."',
  'очерки',
  'интеллектуал.',
  'биографии',
  'густава',
  'шпета'],
 ['использование',
  'дробных',
  'физических',
  'нагрузок',
  'в',
  'статодинамическом',
  'режиме',
  'в',
  'комплексной',
  'реабилитации',
  'больных',
  'ревматоидным',
  'артритом',
  'диссертация',
  '..',
  'кандидата',
  'медицинских',
  'наук',
  '14.00.39'],
 ['отсутствует'],
 ['очерк', 'психологии', 'полиморфной', 'индивидуальности', 'монография'],
 ['факторы,',
  'определяющие',
  'приверженность',
  'к',
  'лечению',
  'больных',
  'ишемической',
  'болезнью',
  'сердца',
  'ди

In [85]:
bm25 = BM25Okapi(tokenized_corpus)

In [86]:
def conv_BM25(x):
    x = x.strip()
    x = x.lower()
    x = x.replace(':','')
    x = x.replace('/','')
    return x.split()

In [90]:
def conv_BM25_get_scores(x):
    x = conv_BM25(x)
    return bm25.get_scores(x)

In [87]:
title = "методы конечных элементов"
tokenized_title = conv_BM25(title)
title_scores = bm25.get_scores(tokenized_title)
title_scores

array([0., 0., 0., ..., 0., 0., 0.])

In [ ]:
items['BM25_code'] = items['title'].apply(conv_BM25_get_scores)

In [ ]:
items.to_hdf("./items.h5") # Сохранить 

In [ ]:
items = pd.read_hdf("./items.h5") # Считать 

In [88]:
bm25.get_top_n(tokenized_title, corpus, n=10)

['методы конечных элементов',
 'применение метода конечных элементов',
 'теория метода конечных элементов',
 'метод конечных элементов  учебное пособие',
 'расчеты машиностроительных конструкций методом конечных элементов  справочник',
 'лекции по методу конечных элементов  учебное пособие',
 'метод конечных элементов в технике  пер. с англ.',
 'решение уравнения переноса методом конечных элементов на неструктурированных треугольных сетках',
 'метод конечных элементов для решения уравнения переноса на неструктурированных тетраэдральных сетках',
 'метод конечных элементов в задачах строительной и непрерывной механики  пер. с англ.']

In [ ]:
items['BM25_code'] = items['title'].apply(conv_BM25, axis=1)